In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [2]:
word = open("names.txt", "r").read().splitlines()

In [3]:
N = torch.zeros((27,27), dtype=torch.int32)
chars = sorted(list(set(''.join(word))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
for w in word:
    chs = ['.'] + list(w) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        x = stoi[c1]
        y = stoi[c2]
        N[x][y] += 1 

In [5]:
# plt.figure(figsize=(16,16))
# plt.imshow(N, cmap='Blues')
# for i in range(27):
#     for j in range(27):
#         chstr = itos[i] + itos[j]
#         plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
#         plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
# plt.axis('off')
# plt.show()

#all this works in the local file "kernel restarted.py" but doesnt on the notebook (for no known reason)

In [9]:
#exemple
p = N[0].float()
p = p /p.sum()
g = torch.Generator().manual_seed(2147483647)
print(p)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])


'j'

In [10]:
P = (N+1).float()
P /= P.sum(1, keepdims=True)
# P = [p / p.sum() for p in N.float()]
# for i in range(27):    
#     print(P[i].sum())

In [ ]:
for i in range (500, 600):
    s = ""
    g = torch.Generator().manual_seed(i)
    p = N[0].float()
    while (True):
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if(itos[ix] == '.'):
            break
        s += itos[ix]
        p = P[ix]
    print(s)

In [12]:
log_likelihood = 0.0
n = 0

for w in word:
#for w in ["andrejq"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood.item()=}')
nll = -log_likelihood.item()
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood.item()=-559951.5625
nll=559951.5625
2.4543562565199477


In [18]:
#training
xs, ys = [], []
for w in word:
    chs = ['.'] + list(w) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        x = stoi[c1]
        y = stoi[c2]
        ys.append(y)
        xs.append(x)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [ ]:
xs

In [ ]:
ys

In [19]:
xenc = F.one_hot(xs, num_classes = 27).float()
xenc.shape

torch.Size([228146, 27])

In [27]:
xenc.shape
w = torch.randn((27,27), requires_grad = True)

In [29]:
logits = xenc @ w
counts = logits.exp()

In [31]:
probs = counts / counts.sum(1, keepdim = True)

In [33]:
probs[0].sum()

tensor(1.0000, grad_fn=<SumBackward0>)

In [ ]:
for k in range (100):
    xenc = F.one_hot(xs, num_classes = 27).float()
    logits = xenc @ w
    counts = logits.exp() # equivalent N
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(w**2).mean()
    print(loss.item())

    w.grad=None
    loss.backward()
    w.data += -50 * w.grad